# 🔬 TorchSim — Atomistic Simulations Powered by PyTorch

---

<div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%); padding: 30px; border-radius: 15px; color: white; margin-bottom: 20px;">

### Welcome to the TorchSim Tutorial Notebook

**TorchSim** is a next-generation, open-source atomistic simulation engine built natively in **PyTorch**.  
By rewriting the core primitives of molecular dynamics in PyTorch, TorchSim achieves up to **100× speedup** over ASE with popular Machine Learning Interatomic Potentials (MLIPs), and uniquely supports **batched simulations** — evolving many systems concurrently on GPUs.

| Feature | Description |
|:--------|:------------|
| 🚀 **GPU-Native** | All computations run natively on CUDA tensors |
| 📦 **Batched Systems** | Simulate 10s–100s of systems simultaneously |
| 🧩 **MLIP Ecosystem** | First-class support for MACE, FairChem, SevenNet, ORB & more |
| 🔄 **ASE/Pymatgen** | Seamless interoperability with the Python materials science stack |
| 📊 **Trajectory I/O** | Built-in HDF5 trajectory recording and analysis |

</div>

### 📖 What You’ll Learn

| Module | Topic | Description |
|:------:|:------|:------------|
| **1** | Setup & Basics | Installation, imports, and core concepts (`SimState`, models) |
| **2** | Bulk MD Simulations | NVT & NPT molecular dynamics of crystalline systems |
| **3** | Structure Optimization | Geometry relaxation with FIRE & cell filters |
| **4** | Surface Energy | Slab construction, static calculations, surface energy formula |
| **5** | Elastic Properties | Full elastic tensor, bulk/shear moduli, Poisson’s ratio |

---

# Module 1 — Setup & Core Concepts

---

<div style="background-color: #e8f4f8; padding: 20px; border-left: 5px solid #2196F3; border-radius: 8px; margin-bottom: 15px;">

### 🎯 Objective

Before we dive into simulations, we need to understand TorchSim’s building blocks:

- **`SimState`** — The central atomistic representation. Think of it as ASE’s `Atoms` but fully tensorized and batch-aware.  
  Every attribute (positions, masses, cell, forces) is a `torch.Tensor`, enabling seamless GPU acceleration.

- **Models** — Any potential energy surface that implements `forward(state) → {energy, forces, stress}`.  
  TorchSim ships with classical potentials (Lennard-Jones, Morse) and wrappers for all major MLIPs.

- **Runners** — Three high-level functions that cover 90% of workflows:  
  `ts.integrate()` (MD), `ts.optimize()` (relaxation), `ts.static()` (single-point energy/force).

</div>

### 1.1 — Installation

Install TorchSim and its dependencies. Uncomment the MACE line if you want to use the MACE universal potential.

In [ ]:
# Install TorchSim (requires Python 3.12+)
# !pip install torch-sim-atomistic

# Optional: Install MACE for the universal foundation model
# !pip install mace-torch

### 1.2 — Imports & Device Setup

In [ ]:
import torch
import torch_sim as ts
import numpy as np
import matplotlib.pyplot as plt
from ase.build import bulk, fcc111, add_adsorbate
from ase.visualize.plot import plot_atoms

# ── Device & Precision ──────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float64  # float64 recommended for MD stability

print(f"TorchSim version : {ts.__version__}")
print(f"PyTorch version  : {torch.__version__}")
print(f"Device           : {device}")
print(f"Precision        : {dtype}")

### 1.3 — Setting Up a Model

We’ll use the **Lennard-Jones** potential throughout this notebook as a lightweight, dependency-free model.  
The same workflows work identically with MACE, FairChem, or any other MLIP — just swap the model.

> **Tip:** To use MACE instead, replace the model block with:
> ```python
> from mace.calculators.foundations_models import mace_mp
> from torch_sim.models.mace import MaceModel
> raw_model = mace_mp(model="small", return_raw_model=True)
> model = MaceModel(model=raw_model, device=device, dtype=dtype)
> ```

In [ ]:
from torch_sim.models.lennard_jones import LennardJonesModel

# Lennard-Jones parameters for Copper (approximate)
lj_model = LennardJonesModel(
    sigma=2.338,       # Å  (equilibrium distance / 2^(1/6))
    epsilon=0.167,     # eV (well depth)
    device=device,
    dtype=dtype,
    compute_forces=True,
    compute_stress=True,
)

print(f"Model      : Lennard-Jones")
print(f"sigma      : {lj_model.sigma:.3f} Å")
print(f"epsilon    : {lj_model.epsilon:.3f} eV")
print(f"cutoff     : {lj_model.cutoff:.3f} Å")

### 1.4 — Creating a SimState

You can create a `SimState` from ASE `Atoms`, pymatgen `Structure`, or raw tensors.  
TorchSim’s `initialize_state()` handles the conversion automatically.

In [ ]:
from torch_sim.state import initialize_state

# Build a bulk Cu FCC crystal using ASE
cu_atoms = bulk("Cu", "fcc", a=3.615, cubic=True)
print(f"ASE Atoms: {len(cu_atoms)} atoms, cell = {cu_atoms.cell.lengths()}")

# Convert to SimState (all tensors, GPU-ready)
state = initialize_state(
    system=cu_atoms,
    device=device,
    dtype=dtype,
)

print(f"\nSimState attributes:")
print(f"  positions     : {state.positions.shape}  (n_atoms, 3)")
print(f"  atomic_numbers: {state.atomic_numbers}")
print(f"  cell          : {state.cell.shape}  (n_systems, 3, 3)")
print(f"  n_atoms       : {state.n_atoms}")
print(f"  n_systems     : {state.n_systems}")
print(f"  device        : {state.device}")

### 1.5 — Quick Static Calculation

Let’s verify everything works with a single-point energy/force evaluation.

In [ ]:
# Single-point calculation
results = ts.static(system=cu_atoms, model=lj_model)

print(f"Energy  : {results[0]['energy'].item():.6f} eV")
print(f"Forces  : {results[0]['forces'].shape} tensor")
print(f"Max |F| : {results[0]['forces'].abs().max().item():.6e} eV/Å")

---

# Module 2 — Bulk Molecular Dynamics

---

<div style="background-color: #fff3e0; padding: 20px; border-left: 5px solid #FF9800; border-radius: 8px; margin-bottom: 15px;">

### 🎯 Objective

Molecular Dynamics (MD) simulates the time evolution of atomic systems by integrating Newton’s equations of motion.  
In this module, we’ll run **NVT** (constant temperature) and **NPT** (constant temperature & pressure) simulations on a bulk copper crystal.

**Key concepts:**

| Ensemble | Fixed Quantities | Thermostat / Barostat | Use Case |
|:--------:|:-----------------|:----------------------|:---------|
| **NVE** | N, V, E | None (microcanonical) | Energy conservation tests |
| **NVT** | N, V, T | Langevin / Nosé-Hoover | Thermal equilibration |
| **NPT** | N, P, T | Langevin barostat | Realistic conditions, density |

TorchSim’s `ts.integrate()` handles everything: initialization of velocities, thermostat coupling, trajectory recording, and automatic batching.

</div>

### 2.1 — Preparing the Supercell

A single unit cell is too small for meaningful MD. We’ll create a **3×3×3 supercell** (108 atoms) to reduce finite-size effects.

In [ ]:
# Build a 3x3x3 FCC copper supercell
cu_bulk = bulk("Cu", "fcc", a=3.615, cubic=True).repeat((3, 3, 3))

print(f"Supercell: {len(cu_bulk)} atoms")
print(f"Cell dimensions: {cu_bulk.cell.lengths()} Å")
print(f"Volume: {cu_bulk.get_volume():.1f} Å³")

# Visualize the structure
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
plot_atoms(cu_bulk, ax=ax, radii=0.4, rotation="10x,10y,0z")
ax.set_title("Cu FCC 3×3×3 Supercell (108 atoms)", fontsize=13, fontweight="bold")
ax.set_axis_off()
plt.tight_layout()
plt.show()

### 2.2 — NVT Molecular Dynamics (Langevin Thermostat)

The **Langevin thermostat** couples the system to a heat bath via stochastic forces.  
It’s robust, easy to use, and ideal for equilibration.

In [ ]:
from torch_sim.trajectory import TrajectoryReporter

# ── Define trajectory reporter to record properties ─────────────────
prop_calculators = {
    10: {
        "potential_energy": lambda state: state.energy,
        "temperature": lambda state: ts.calc_temperature(
            masses=state.masses,
            velocities=state.velocities,
        ),
        "kinetic_energy": lambda state: ts.calc_kinetic_energy(
            masses=state.masses,
            velocities=state.velocities,
        ),
    }
}

reporter_nvt = TrajectoryReporter(
    filenames="nvt_trajectory.h5",
    state_frequency=50,
    prop_calculators=prop_calculators,
)

# ── Run NVT MD ──────────────────────────────────────────────────────
print("Running NVT MD at 300 K ...")

final_state_nvt = ts.integrate(
    system=cu_bulk,
    model=lj_model,
    integrator=ts.Integrator.nvt_langevin,
    n_steps=2000,
    temperature=300,        # Kelvin
    timestep=0.002,         # picoseconds (2 fs)
    trajectory_reporter=reporter_nvt,
    pbar=True,
)

print(f"\nFinal energy: {final_state_nvt.energy.item():.4f} eV")

### 2.3 — Analyzing the NVT Trajectory

Let’s read the saved trajectory and plot how the system equilibrates.

In [ ]:
from torch_sim.trajectory import TorchSimTrajectory

with TorchSimTrajectory("nvt_trajectory.h5", mode="r") as traj:
    pe = traj.get_array("potential_energy").numpy()
    ke = traj.get_array("kinetic_energy").numpy()
    temp = traj.get_array("temperature").numpy()
    steps_pe = traj.get_steps("potential_energy")

total_energy = pe + ke
time_ps = np.array(steps_pe) * 0.002  # convert steps to ps

# ── Plot ─────────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 4.5))

# Temperature
axes[0].plot(time_ps, temp, color="#e74c3c", alpha=0.7, lw=0.8)
axes[0].axhline(300, color="black", ls="--", lw=1, label="Target 300 K")
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("Temperature (K)")
axes[0].set_title("Temperature", fontweight="bold")
axes[0].legend()

# Energies
axes[1].plot(time_ps, pe, label="Potential", color="#3498db", alpha=0.8, lw=0.8)
axes[1].plot(time_ps, ke, label="Kinetic", color="#e67e22", alpha=0.8, lw=0.8)
axes[1].plot(time_ps, total_energy, label="Total", color="#2ecc71", alpha=0.8, lw=0.8)
axes[1].set_xlabel("Time (ps)")
axes[1].set_ylabel("Energy (eV)")
axes[1].set_title("Energy Components", fontweight="bold")
axes[1].legend()

# Energy fluctuation
axes[2].plot(time_ps, (total_energy - total_energy.mean()) * 1000,
             color="#9b59b6", alpha=0.7, lw=0.8)
axes[2].set_xlabel("Time (ps)")
axes[2].set_ylabel("\u0394E (meV)")
axes[2].set_title("Total Energy Fluctuation", fontweight="bold")

for ax in axes:
    ax.grid(True, alpha=0.3)

fig.suptitle("NVT Langevin MD — Cu Bulk at 300 K", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

print(f"Mean temperature : {temp.mean():.1f} ± {temp.std():.1f} K")
print(f"Mean PE          : {pe.mean():.4f} ± {pe.std():.4f} eV")

### 2.4 — NPT Molecular Dynamics (Langevin Barostat)

<div style="background-color: #f3e5f5; padding: 15px; border-left: 5px solid #9C27B0; border-radius: 8px; margin-bottom: 10px;">

**NPT** simulations allow both the **temperature** and **pressure** to fluctuate around target values,  
while the simulation cell volume adjusts dynamically. This is the closest analogue to real experimental conditions  
(constant temperature and atmospheric pressure).

</div>

In [ ]:
reporter_npt = TrajectoryReporter(
    filenames="npt_trajectory.h5",
    state_frequency=50,
    prop_calculators={
        10: {
            "potential_energy": lambda state: state.energy,
            "temperature": lambda state: ts.calc_temperature(
                masses=state.masses,
                velocities=state.velocities,
            ),
        }
    },
)

print("Running NPT MD at 300 K, 0 bar ...")

final_state_npt = ts.integrate(
    system=cu_bulk,
    model=lj_model,
    integrator=ts.Integrator.npt_langevin,
    n_steps=2000,
    temperature=300,        # K
    timestep=0.002,         # ps
    trajectory_reporter=reporter_npt,
    pbar=True,
)

# Compare initial vs final cell
initial_vol = cu_bulk.get_volume()
final_vol = final_state_npt.volume.item()
print(f"\nVolume change: {initial_vol:.1f} → {final_vol:.1f} Å³ ({(final_vol/initial_vol - 1)*100:+.2f}%)")

### 2.5 — Batched MD: Simulating Multiple Systems at Once

<div style="background-color: #e0f2f1; padding: 15px; border-left: 5px solid #009688; border-radius: 8px; margin-bottom: 10px;">

**TorchSim’s killer feature**: run multiple independent simulations in a single call.  
This is especially powerful on GPUs where the overhead of launching kernels dominates —  
batching amortizes that cost across many systems.

Here we run **5 replicas** at different temperatures simultaneously.

</div>

In [ ]:
# Create 5 identical starting configurations
temperatures = [100, 200, 300, 500, 800]  # K
systems = [cu_bulk.copy() for _ in temperatures]

print(f"Running batched NVT MD for {len(temperatures)} systems ...")
print(f"Temperatures: {temperatures} K\n")

final_states = ts.integrate(
    system=systems,
    model=lj_model,
    integrator=ts.Integrator.nvt_langevin,
    n_steps=1000,
    temperature=temperatures,  # per-system temperature!
    timestep=0.002,
    pbar=True,
)

# Extract per-system results
for i, T in enumerate(temperatures):
    sub_state = final_states[i]
    measured_T = ts.calc_temperature(
        masses=sub_state.masses,
        velocities=sub_state.velocities,
    ).item()
    print(f"  T_target = {T:4d} K  |  T_measured = {measured_T:7.1f} K  |  E = {sub_state.energy.item():.4f} eV")

---

# Module 3 — Structure Optimization & Relaxation

---

<div style="background-color: #e8f5e9; padding: 20px; border-left: 5px solid #4CAF50; border-radius: 8px; margin-bottom: 15px;">

### 🎯 Objective

In computational materials science, **geometry optimization** (relaxation) finds the nearest local energy minimum —  
the configuration where all forces on atoms vanish. This is a prerequisite for almost every property calculation.

TorchSim provides several optimizers:

| Optimizer | Method | Best For |
|:----------|:-------|:---------|
| **FIRE** | Fast Inertial Relaxation Engine | General-purpose, robust |
| **BFGS** | Quasi-Newton | Smooth energy landscapes |
| **L-BFGS** | Limited-memory BFGS | Large systems |
| **Gradient Descent** | Steepest descent | Simple, debugging |

**Cell filters** allow simultaneous relaxation of atomic positions *and* unit cell shape/volume:
- `ts.CellFilter.frechet` — Operates in log-space, better numerical properties
- `ts.CellFilter.unit` — Standard unit cell optimization

</div>

### 3.1 — Relaxing a Perturbed Crystal

In [ ]:
# Start with a slightly perturbed structure
cu_perturbed = bulk("Cu", "fcc", a=3.615, cubic=True).repeat((2, 2, 2))

# Add random perturbations to atomic positions
rng = np.random.default_rng(42)
cu_perturbed.positions += rng.normal(0, 0.1, cu_perturbed.positions.shape)

# Also strain the cell slightly
cu_perturbed.cell *= 1.02  # 2% volumetric expansion

# ── Static calc on perturbed structure ───────────────────────────────
res_before = ts.static(system=cu_perturbed, model=lj_model)
print(f"Before relaxation:")
print(f"  Energy  : {res_before[0]['energy'].item():.6f} eV")
print(f"  Max |F| : {res_before[0]['forces'].abs().max().item():.4f} eV/Å")

### 3.2 — FIRE Optimization with Cell Relaxation

In [ ]:
# ── Relax positions + cell with FIRE optimizer ──────────────────────
print("Relaxing with FIRE + Frechet cell filter ...\n")

relaxed_state = ts.optimize(
    system=cu_perturbed,
    model=lj_model,
    optimizer=ts.Optimizer.fire,
    convergence_fn=ts.generate_force_convergence_fn(force_tol=1e-4),
    max_steps=5000,
    init_kwargs=dict(cell_filter=ts.CellFilter.frechet),
    pbar=True,
)

# ── Compare ─────────────────────────────────────────────────────────
res_after = ts.static(system=relaxed_state.to_atoms(), model=lj_model)

print(f"\nAfter relaxation:")
print(f"  Energy  : {res_after[0]['energy'].item():.6f} eV")
print(f"  Max |F| : {res_after[0]['forces'].abs().max().item():.2e} eV/Å")
print(f"\n  ΔE = {res_after[0]['energy'].item() - res_before[0]['energy'].item():.6f} eV")

### 3.3 — Batched Optimization: Relaxing Multiple Structures

Just like MD, optimization can be batched. This is perfect for high-throughput screening.

In [ ]:
# Create structures with different lattice constants
lattice_constants = [3.4, 3.5, 3.6, 3.7, 3.8]
structures = [bulk("Cu", "fcc", a=a, cubic=True) for a in lattice_constants]

print(f"Relaxing {len(structures)} structures in a single batch ...\n")

relaxed_batch = ts.optimize(
    system=structures,
    model=lj_model,
    optimizer=ts.Optimizer.fire,
    convergence_fn=ts.generate_force_convergence_fn(force_tol=1e-3),
    max_steps=2000,
    init_kwargs=dict(cell_filter=ts.CellFilter.frechet),
    pbar=True,
)

# All structures should converge to the same equilibrium
print(f"\n{'a_init (Å)':>10} | {'a_final (Å)':>12} | {'E (eV)':>10}")
print("-" * 40)

atoms_list = relaxed_batch.to_atoms()
for i, a0 in enumerate(lattice_constants):
    a_final = atoms_list[i].cell.lengths()[0]
    res = ts.static(system=atoms_list[i], model=lj_model)
    e = res[0]['energy'].item()
    print(f"{a0:10.2f} | {a_final:12.4f} | {e:10.6f}")

---

# Module 4 — Surface Energy Calculations

---

<div style="background-color: #fce4ec; padding: 20px; border-left: 5px solid #E91E63; border-radius: 8px; margin-bottom: 15px;">

### 🎯 Objective

The **surface energy** (γ) quantifies the energetic cost of creating a surface by cleaving a crystal.  
It is a fundamental quantity in materials science, governing:

- Crystal growth morphology (Wulff construction)
- Catalytic activity
- Adhesion and wetting
- Nanoparticle stability

**The formula:**

$$\gamma = \frac{E_{\text{slab}} - N \cdot E_{\text{bulk/atom}}}{2A}$$

where:
- $E_{\text{slab}}$ = total energy of the slab
- $N$ = number of atoms in the slab
- $E_{\text{bulk/atom}}$ = energy per atom of the bulk crystal
- $A$ = surface area (factor of 2 because a slab has two surfaces)

**Workflow:**
1. Relax the bulk unit cell → get $E_{\text{bulk/atom}}$
2. Build slabs of various thicknesses with vacuum
3. Relax slab atoms (fix cell) → get $E_{\text{slab}}$
4. Compute $\gamma$ and check convergence with slab thickness

</div>

### 4.1 — Step 1: Bulk Reference Energy

In [ ]:
# Relax bulk Cu to get the reference energy per atom
cu_unit = bulk("Cu", "fcc", a=3.615, cubic=True)

relaxed_bulk = ts.optimize(
    system=cu_unit,
    model=lj_model,
    optimizer=ts.Optimizer.fire,
    convergence_fn=ts.generate_force_convergence_fn(force_tol=1e-5),
    max_steps=5000,
    init_kwargs=dict(cell_filter=ts.CellFilter.frechet),
)

bulk_result = ts.static(system=relaxed_bulk.to_atoms(), model=lj_model)
E_bulk_total = bulk_result[0]['energy'].item()
n_bulk = len(cu_unit)
E_bulk_per_atom = E_bulk_total / n_bulk

print(f"Bulk Cu (relaxed):")
print(f"  Total energy   : {E_bulk_total:.6f} eV  ({n_bulk} atoms)")
print(f"  Energy/atom    : {E_bulk_per_atom:.6f} eV/atom")
print(f"  Lattice const  : {relaxed_bulk.to_atoms()[0].cell.lengths()[0]:.4f} Å")

### 4.2 — Step 2: Build & Relax Surface Slabs

We’ll compute the (111) surface energy for Cu at various slab thicknesses to check convergence.

In [ ]:
from ase.build import fcc111

# Get relaxed lattice constant
a_relaxed = relaxed_bulk.to_atoms()[0].cell.lengths()[0]

# Build slabs with different number of layers
layer_counts = [3, 4, 5, 6, 7, 8]
vacuum = 15.0  # Å of vacuum on each side

slabs = []
for n_layers in layer_counts:
    slab = fcc111("Cu", size=(2, 2, n_layers), a=a_relaxed, vacuum=vacuum, periodic=True)
    slabs.append(slab)
    print(f"  {n_layers} layers: {len(slab):3d} atoms, "
          f"cell = [{slab.cell.lengths()[0]:.2f}, {slab.cell.lengths()[1]:.2f}, {slab.cell.lengths()[2]:.2f}] Å")

# Visualize thinnest and thickest slabs
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
for ax, slab, n in zip(axes, [slabs[0], slabs[-1]], [layer_counts[0], layer_counts[-1]]):
    plot_atoms(slab, ax=ax, radii=0.4, rotation="-90x,0y,0z")
    ax.set_title(f"Cu(111) — {n} layers", fontsize=12, fontweight="bold")
    ax.set_axis_off()
plt.tight_layout()
plt.show()

### 4.3 — Step 3: Compute Surface Energies

In [ ]:
# ── Relax all slabs (positions only, cell fixed) in a batch ─────────
print("Relaxing all slabs (batched) ...\n")

relaxed_slabs = ts.optimize(
    system=slabs,
    model=lj_model,
    optimizer=ts.Optimizer.fire,
    convergence_fn=ts.generate_force_convergence_fn(force_tol=1e-3),
    max_steps=3000,
    pbar=True,
)

# ── Compute surface energy for each slab ────────────────────────────
slab_atoms = relaxed_slabs.to_atoms()
slab_results = ts.static(system=slab_atoms, model=lj_model)

surface_energies = []
eV_per_A2_to_J_per_m2 = 16.0218  # conversion factor

print(f"{'Layers':>6} | {'N_atoms':>7} | {'E_slab (eV)':>12} | {'Area (Å²)':>10} | {'γ (eV/Å²)':>11} | {'γ (J/m²)':>9}")
print("-" * 72)

for i, n_layers in enumerate(layer_counts):
    E_slab = slab_results[i]['energy'].item()
    N = len(slab_atoms[i])
    # Surface area = |a1 x a2| (cross product of first two cell vectors)
    cell = slab_atoms[i].cell
    area = np.linalg.norm(np.cross(cell[0], cell[1]))
    
    gamma = (E_slab - N * E_bulk_per_atom) / (2 * area)  # eV/Å²
    gamma_SI = gamma * eV_per_A2_to_J_per_m2             # J/m²
    surface_energies.append(gamma_SI)
    
    print(f"{n_layers:6d} | {N:7d} | {E_slab:12.4f} | {area:10.2f} | {gamma:11.6f} | {gamma_SI:9.3f}")

### 4.4 — Surface Energy Convergence Plot

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(layer_counts, surface_energies, "o-", color="#E91E63", lw=2, markersize=8)
ax.axhline(surface_energies[-1], color="gray", ls="--", alpha=0.5, label=f"Converged: {surface_energies[-1]:.3f} J/m²")

ax.set_xlabel("Number of Layers", fontsize=12)
ax.set_ylabel("Surface Energy γ (J/m²)", fontsize=12)
ax.set_title("Cu(111) Surface Energy Convergence", fontsize=14, fontweight="bold")
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xticks(layer_counts)

plt.tight_layout()
plt.show()

print(f"\nConverged surface energy: γ = {surface_energies[-1]:.3f} J/m²")
print(f"(Experimental value for Cu(111): ~1.79 J/m² — LJ is a rough approximation)")

---

# Module 5 — Elastic Properties

---

<div style="background-color: #ede7f6; padding: 20px; border-left: 5px solid #673AB7; border-radius: 8px; margin-bottom: 15px;">

### 🎯 Objective

The **elastic tensor** $C_{ij}$ describes how a material responds to small deformations.  
From it we can extract fundamental mechanical properties:

| Property | Symbol | Meaning |
|:---------|:------:|:--------|
| Bulk modulus | $K$ | Resistance to uniform compression |
| Shear modulus | $G$ | Resistance to shape change |
| Poisson’s ratio | $\nu$ | Lateral strain vs. axial strain |
| Pugh’s ratio | $K/G$ | Ductility indicator (>1.75 = ductile) |

**TorchSim’s approach:**  
1. Apply a set of small strain deformations to the relaxed crystal  
2. Compute the stress tensor for each deformation (batched!)  
3. Fit the stress-strain relation to extract $C_{ij}$  
4. Use Voigt-Reuss-Hill averaging for polycrystalline moduli

All of this is wrapped in a clean two-function API: `calculate_elastic_tensor()` + `calculate_elastic_moduli()`.

</div>

### 5.1 — Prepare a Relaxed Structure

In [ ]:
from torch_sim.state import initialize_state

# Use the already relaxed bulk from Module 4
cu_relaxed_atoms = relaxed_bulk.to_atoms()[0]
cu_elastic_state = initialize_state(
    system=cu_relaxed_atoms,
    device=device,
    dtype=dtype,
)

print(f"Structure for elastic calculation:")
print(f"  Formula : Cu{len(cu_relaxed_atoms)}")
print(f"  a       : {cu_relaxed_atoms.cell.lengths()[0]:.4f} Å")
print(f"  Volume  : {cu_relaxed_atoms.get_volume():.2f} Å³")

### 5.2 — Calculate the Full Elastic Tensor

In [ ]:
from torch_sim.elastic import calculate_elastic_tensor, calculate_elastic_moduli

print("Computing elastic tensor (applying strain deformations) ...\n")

C = calculate_elastic_tensor(
    state=cu_elastic_state,
    model=lj_model,
    max_strain_normal=0.01,    # 1% normal strain
    max_strain_shear=0.06,     # 6% shear strain
    n_deform=5,                # 5 deformation magnitudes per mode
)

# Display the 6x6 elastic tensor (Voigt notation)
print("Elastic Tensor C_ij (GPa):")
print("=" * 60)
C_GPa = C.numpy() * 160.2176634  # eV/Å³ -> GPa
labels = ["C11", "C22", "C33", "C44", "C55", "C66"]
for i in range(6):
    row = "  ".join(f"{C_GPa[i,j]:8.2f}" for j in range(6))
    print(f"  [{row}]")
print()

### 5.3 — Extract Elastic Moduli

In [ ]:
# Calculate Voigt-Reuss-Hill averaged moduli
bulk_mod, shear_mod, poisson, pugh = calculate_elastic_moduli(C)

# Convert to GPa
eV_A3_to_GPa = 160.2176634
K = bulk_mod.item() * eV_A3_to_GPa
G = shear_mod.item() * eV_A3_to_GPa
nu = poisson.item()
pugh_ratio = pugh.item()

print("\n" + "=" * 50)
print("  ELASTIC PROPERTIES OF Cu (LJ potential)")
print("=" * 50)
print(f"  Bulk modulus  (K)  : {K:8.2f} GPa")
print(f"  Shear modulus (G)  : {G:8.2f} GPa")
print(f"  Young's modulus    : {9*K*G/(3*K+G):8.2f} GPa")
print(f"  Poisson's ratio    : {nu:8.4f}")
print(f"  Pugh's ratio (K/G) : {pugh_ratio:8.4f}")
print(f"  Ductility          : {'Ductile' if pugh_ratio > 1.75 else 'Brittle'}")
print("=" * 50)

### 5.4 — Visualizing Elastic Properties

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ── Bar chart of moduli ─────────────────────────────────────────────
E_young = 9 * K * G / (3 * K + G)
props = ["Bulk (K)", "Shear (G)", "Young's (E)"]
values = [K, G, E_young]
colors = ["#673AB7", "#9C27B0", "#BA68C8"]

bars = axes[0].bar(props, values, color=colors, width=0.5, edgecolor="white", linewidth=1.5)
for bar, val in zip(bars, values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f"{val:.1f}", ha="center", va="bottom", fontweight="bold", fontsize=11)
axes[0].set_ylabel("Modulus (GPa)", fontsize=12)
axes[0].set_title("Elastic Moduli", fontsize=13, fontweight="bold")
axes[0].grid(axis="y", alpha=0.3)

# ── Heatmap of elastic tensor ──────────────────────────────────────
im = axes[1].imshow(C_GPa, cmap="RdBu_r", aspect="equal",
                     vmin=-np.abs(C_GPa).max(), vmax=np.abs(C_GPa).max())
for i in range(6):
    for j in range(6):
        axes[1].text(j, i, f"{C_GPa[i,j]:.1f}", ha="center", va="center",
                     fontsize=8, color="white" if abs(C_GPa[i,j]) > 0.5*np.abs(C_GPa).max() else "black")

axes[1].set_xticks(range(6))
axes[1].set_yticks(range(6))
voigt = ["1", "2", "3", "4", "5", "6"]
axes[1].set_xticklabels(voigt)
axes[1].set_yticklabels(voigt)
axes[1].set_xlabel("j")
axes[1].set_ylabel("i")
axes[1].set_title("Elastic Tensor C$_{ij}$ (GPa)", fontsize=13, fontweight="bold")
plt.colorbar(im, ax=axes[1], shrink=0.8, label="GPa")

fig.suptitle("Cu Elastic Properties (LJ Potential)", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

---

# Summary & Next Steps

---

<div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%); padding: 25px; border-radius: 15px; color: white;">

### ✅ What We Covered

| Module | Technique | Key TorchSim API |
|:------:|:----------|:------------------|
| **1** | Setup & SimState | `initialize_state()`, `ts.static()` |
| **2** | NVT/NPT MD, Batched MD | `ts.integrate()`, `TrajectoryReporter` |
| **3** | FIRE Optimization, Cell Relaxation | `ts.optimize()`, `CellFilter.frechet` |
| **4** | Surface Energy Calculation | Slab construction + `ts.static()` |
| **5** | Elastic Tensor & Moduli | `calculate_elastic_tensor()`, `calculate_elastic_moduli()` |

### 🚀 Where to Go from Here

- **Switch to MLIPs**: Replace `LennardJonesModel` with `MaceModel` for DFT-quality results
- **Phonon Calculations**: Use TorchSim’s Phonopy integration for lattice dynamics
- **Monte Carlo**: Explore `torch_sim.monte_carlo` for swap MC and hybrid methods
- **Large-Scale Screening**: Leverage batched optimization to relax 1000s of structures on GPU
- **Custom Workflows**: Combine the low-level `init_fn`/`step_fn` API for custom integrators

### 📚 Resources

- [TorchSim Documentation](https://torchsim.github.io/torch-sim/)
- [GitHub Repository](https://github.com/TorchSim/torch-sim)
- [TorchSim Paper (arXiv)](https://arxiv.org/abs/2508.06628)

</div>

In [ ]:
# ── Cleanup temporary trajectory files ──────────────────────────────
import os
for f in ["nvt_trajectory.h5", "npt_trajectory.h5"]:
    if os.path.exists(f):
        os.remove(f)
        print(f"Removed {f}")

print("\nDone! Happy simulating! 🔬")